In [15]:
# TODO: move to requirements
!pip install jhub-client

In [16]:
# define some useful variables for following steps
import json
import os
from pathlib import Path
import requests
import urllib3

print("Setup configuration parameters...")

VERIFY_SSL = True if 'DISABLE_VERIFY_SSL' not in os.environ else False
if not VERIFY_SSL:
    urllib3.disable_warnings()  # disable warnings for using https without certificate verification enabled

HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca")
assert PAVICS_HOST != "", "Invalid PAVICS HOST value."

TEST_DISABLE_CLEANUP = str(os.getenv("TEST_DISABLE_CLEANUP", False)).lower() in ["true", "1", "on"]

COWBIRD_URL = f"https://{PAVICS_HOST}/cowbird"
JUPYTERHUB_URL = f"https://{PAVICS_HOST}/jupyter/hub"
MAGPIE_URL = f"https://{PAVICS_HOST}/magpie"

def get_credentials(var_name):
    value = os.getenv(var_name)
    if not value:
        raise ValueError("Missing test admin credentials `{}` to run tests.".format(var_name))
    return value

TEST_MAGPIE_ADMIN_USERNAME = get_credentials("TEST_MAGPIE_ADMIN_USERNAME")
TEST_MAGPIE_ADMIN_PASSWORD = get_credentials("TEST_MAGPIE_ADMIN_PASSWORD")

TEST_USER = "testcowbirdjupyter"
TEST_PASSWORD = "qwertyqwerty"

GEOSERVER_TEST_DATA_DIR = os.getenv("GEOSERVER_TEST_DATA_DIR", "/geoserver-test-data")

print("  Verify SSL : {}".format(VERIFY_SSL))
print("  Will use JupyterHub URL:   [{}]".format(JUPYTERHUB_URL))

Setup configuration parameters...
  Verify SSL : False
  Will use JupyterHub URL:   [https://172.17.0.1/jupyter/hub]


In [17]:
def cleanup_test_data(skip_fail=True):
    """Cleanup test elements created during the test.

        Only raise at the complete end to attempt removal of as much element as possible in case of failures.
        """
    if TEST_DISABLE_CLEANUP:
        print("WARNING - Skipping test data cleanup because of TEST_DISABLE_CLEANUP option!")
        return

    failed = False

    # Cleanup Magpie users and resources
    magpie_items = [("users", "MAGPIE_TEST_USER", TEST_USER)]
    # Apply cleanup of Magpie items
    for item_type, item_name, item_value in magpie_items:
        try:
            _path = "{}/{}/{}".format(MAGPIE_URL, item_type, item_value)
            _resp = magpie_admin_session.delete(_path)
            _head = "Cleanup of {} :".format(item_name)
            if _resp.status_code == 200:
                print(_head, "OK")
            elif _resp.status_code == 404:
                print(_head, "WARNING - does not exist (maybe it failed to be created in the first place?)")
                failed = True  # most definitely the test is already failing at this point anyway
            else:
                print(_head, "ERROR - could not be removed")
                failed = True
        except Exception as exc:
            print("Magpie cleanup raised: [{}]".format(exc))
            failed = True
    if not skip_fail and failed:
        raise ValueError("Could not cleanup all test data.")

In [18]:
def response_msg(message, response, is_json=True):
    """Append useful response details to provided message."""
    _body = response.text
    _detail = "<unknown>"
    if is_json:
        try:
            _body = response.json()
            _detail = _body.get("detail", _body.get("message", "<unknown>"))
        except json.JSONDecodeError:
            # ignore and revert to text body since it could not be parsed as JSON
            _body = response.text
    return "{} Response replied with ({}) [{}]\nContent: {}\n\n".format(message, response.status_code, _detail, _body)

In [19]:
# Check if Cowbird is rolling, else display error
assert requests.get(COWBIRD_URL, verify=VERIFY_SSL).status_code == 200, "Cowbird is not available, but should be actiavted for this test."

In [20]:
# Create user on Magpie

In [21]:
def magpie_signin(user_name, password):
    signin_url = f"{MAGPIE_URL}/signin"
    data = {"user_name": user_name, "password": password}
    try:
        resp = requests.request(url=signin_url, headers=HEADERS, method="POST", json=data, timeout=10, verify=VERIFY_SSL)
    except Exception as exc:
        raise RuntimeError(f"Failed to sign in to Magpie (url: `{signin_url}`) with user `{data['user_name']}`. "
                           f"Exception : {exc}. ")
    if resp.status_code != 200:
        raise RuntimeError(f"Unexpected response while trying to sign in to Magpie with user `{user_name}` : {resp.text}")
    return resp

In [22]:
magpie_admin_session = requests.Session()
magpie_admin_session.verify = VERIFY_SSL
magpie_admin_session.headers = HEADERS
magpie_admin_session.cookies = magpie_signin(TEST_MAGPIE_ADMIN_USERNAME, TEST_MAGPIE_ADMIN_PASSWORD).cookies

test_user_session = requests.Session()
test_user_session.verify = VERIFY_SSL
test_user_session.headers = HEADERS

In [24]:
def create_magpie_user(user_name, password, session):
    resp = magpie_admin_session.post(
        url=f"{MAGPIE_URL}/users",
        json={
            "user_name": user_name,
            "email": f"{user_name}@user.com",
            "password": password,
            "group_name": "users"},
        allow_redirects=False)
    if resp.status_code != 201:
        cleanup_test_data()
        raise ValueError(response_msg("\nCould not create test user [{}]".format(user_name), resp))
    session.cookies = magpie_signin(user_name, password).cookies

create_magpie_user(TEST_USER, TEST_PASSWORD, test_user_session)

In [31]:
# Add geoserver shapefiles to user workspace ???
# for filename in os.listdir(GEOSERVER_TEST_DATA_DIR):
#     shutil.copy2(os.path.join(GEOSERVER_TEST_DATA_DIR, filename), # TARGET_PATH^??????

#     Faire un 2e shapefile (copie) qui est read-only, pour que Magpie set les permissions directement

res = !ls /data
res2 = !ls /data/user_workspaces

raise ValueError(res, res2)

ValueError: (['datasets', 'frontend_persist', 'geoserver', 'jupyterhub_user_data', 'magpie_persist', 'mongodb_cowbird_persist', 'ncml', 'user_workspaces'], ['admin', 'cwtest', 'public', 'testcowbirdjupyter', 'test-user-0708ff64-61ba-48bd-ad06-6f5c1c58097d', 'test-user-20f934e3-0099-4b22-af70-04d9896a8328', 'test-user-ddb725f4-cd16-47a3-af20-b2395531310b', 'test-user-no-perm-0ad33a01-7df0-4c74-ab7b-d3ab0e356370', 'test-user-no-perm-8754ddb7-2647-4f92-bd33-aebf78796a68', 'test-user-no-perm-d5eef900-55b6-4157-b19c-669ada56ccd5'])

In [ ]:
# Change Geoserver permissions

In [ ]:
# Add wps-outputs
# Créer un fichier ici, et le copier dans les paths de wpsoutputs?? Pour que les hardlinks soient générés correctement

# public
# user specific

In [25]:
# Get JupyterLab user token
resp = magpie_admin_session.post(
    f"{JUPYTERHUB_URL}/api/users/{TEST_USER}/tokens", 
    headers={"Authorization": "Token admin-token", "Accept": "application/json"})
if resp.status_code != 201:
    raise ValueError(response_msg("\nFailed to start JupyterLab Docker container.", resp))
os.environ["JUPYTERHUB_API_TOKEN"] = resp.json()["token"]

In [26]:
!jhubctl run --hub "https://localhost/jupyter/" --auth-type token -u testcowbirdjupyter -n resources/user_test_cowbird_jupyter.ipynb \
  --no-verify-ssl --output-filename notebook_results.txt --stop-server --validate

INFO:jhub_client.api:creating cluster username=testcowbirdjupyter user_options={}
INFO:jhub_client.api:created server for username=testcowbirdjupyter with user_options={}
INFO:jhub_client.api:created kernel_spec=python3 kernel=d9f1df06-3526-4d00-8e81-766981d2eab0 for jupyter
INFO:jhub_client.api:deleted kernel=d9f1df06-3526-4d00-8e81-766981d2eab0 for jupyter
INFO:jhub_client.api:deleted server for username=testcowbirdjupyter


In [ ]:
# Create JupyterLab Docker container
# resp = magpie_admin_session.post(
#     f"{JUPYTERHUB_URL}/api/users/{TEST_USER}/server", 
#     headers={"Authorization": "Token admin-token", "Accept": "application/json"})
# if resp.status_code != 201:
#     raise ValueError(response_msg("\nFailed to start JupyterLab Docker container.", resp))

In [ ]:
# Execute nbval on the user test on the spawned container
# !docker exec -it jupyter-{TEST_USER} py.test --nbval /notebook_dir/test_cowbird_jupyter_test_user.ipynb

In [ ]:
# Test delete user on Magpie
# Check if workspace folder and geoserver workspace are deleted

In [ ]:
# Clean-up (if not already deleted)
# Magpie resources (Geoserver, wps-outputs)
# Magpie user
# Filesystem (workspace, wps-outputs de test public/user, user notebook_dir in jupyterhub_user_data)
# geoserver workspace